In [ ]:
%matplotlib inline

# Sampling with repetitions


In [ ]:
from __future__ import annotations

from gemseo import configure_logger
from gemseo.algos.design_space import DesignSpace
from gemseo.algos.parameter_space import ParameterSpace
from gemseo.disciplines.analytic import AnalyticDiscipline
from gemseo.settings.doe import OT_LHS_Settings
from matplotlib import pyplot as plt
from numpy import load
from numpy import save
from numpy import stack
from numpy import vstack

from gemseo_umdo.formulations.sampling_settings import Sampling_Settings
from gemseo_umdo.scenarios.umdo_scenario import UMDOScenario

configure_logger()

Firstly,
we instantiate the disciplines of the Sellar problem:



In [ ]:
system = AnalyticDiscipline({
    "obj": "x**2 + z2 + y1**2 + exp(-y2)",
    "c1": "3.16 - y1 ** 2",
    "c2": "y2 - 24.0",
})
disc1 = AnalyticDiscipline({"y1": "(z1**2 + z2 + x - a*y2)**0.5"})
disc2 = AnalyticDiscipline({"y2": "2/10*log(1+exp(10*y1))-y1-2/10*log(2) + z1 + z2"})

as well as a function to instantiate the design space:



In [ ]:
def create_design_space() -> DesignSpace:
    """Create the design space for the Sellar problem."""
    design_space = DesignSpace()
    design_space.add_variable("x", lower_bound=0.0, upper_bound=10.0, value=1.0)
    design_space.add_variable("z1", lower_bound=-10, upper_bound=10.0, value=4.0)
    design_space.add_variable("z2", lower_bound=0.0, upper_bound=10.0, value=3.0)
    return design_space

Secondly,
we define the uncertain space:



In [ ]:
uncertain_space = ParameterSpace()

with an uncertainty over the constant `"a"`:



In [ ]:
uncertain_space.add_random_variable(
    "a", "OTTriangularDistribution", minimum=0.1, maximum=0.3, mode=0.2
)

Then,
we build 10 [UMDOScenario][gemseo_umdo.scenarios.umdo_scenario.UMDOScenario]
to minimize a sampling-based estimation
of the expectation $\mathbb{E}[obj]$
and store the history of the design values:



In [ ]:
x_hist = []
for i in range(10):
    print(i)
    scenario = UMDOScenario(
        [system, disc1, disc2],
        "obj",
        create_design_space(),
        uncertain_space,
        "Mean",
        formulation_name="MDF",
        statistic_estimation_settings=Sampling_Settings(
            doe_algo_settings=OT_LHS_Settings(n_samples=100, seed=i + 1)
        ),
    )
    scenario.add_constraint("c1", "Margin", factor=3.0)
    scenario.add_constraint("c2", "Margin", factor=3.0)
    scenario.execute(algo_name="NLOPT_COBYLA", max_iter=100)
    x_hist.append(
        vstack(scenario.formulation.optimization_problem.database.get_x_vect_history())
    )

Lastly,
we plot the variability of the optimization history with boxplots:



In [ ]:
print(x_hist)
x_hist = stack(x_hist)
save("x_hist.npy", x_hist)
x_hist = load("x_hist.npy")
plt.boxplot(x_hist[:, :, 0])
plt.savefig("hist.png")